In [ ]:
import sys
sys.path.append('../src/')

%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np


In [ ]:
PATH = Path.cwd().parent.joinpath('data', 'voi')

In [ ]:
# Import vessel data

dfs = []

for file in PATH.joinpath('gfw_data', 'vessel_info').glob('*.csv'):
    name = file.stem.split('-')[0]
    dfs.append(pd.read_csv(file))

df = pd.concat(dfs)
len(df)


In [ ]:
# Clean ownership data

owner = df[['id', 'owner']].copy()
owner.owner = owner.owner.str.split('|')
owner = owner.explode('owner')
owner['start_date'] = owner.owner.apply(lambda x: x.split(' (')[1] if x is not np.nan else x)
owner['end_date'] = owner.start_date.apply(lambda x: x.split('Z-')[1] if x is not np.nan else x)
owner.start_date = owner.start_date.apply(lambda x: pd.to_datetime(x[0:10]) if x is not np.nan else x)
owner.end_date = owner.end_date.apply(lambda x: pd.to_datetime(x[0:10]) if x is not np.nan else x)
owner.owner = owner.owner.apply(lambda x: x.split(' (')[0].split('-')[0] if x is not np.nan else x)
owner.head()

In [ ]:
owner.groupby(owner.end_date.dt.year).agg(aantal_bedrijven=('owner', lambda x: x.nunique()))

In [ ]:
owner.groupby('id').owner.count().sort_values(ascending=False).nlargest(10)

In [ ]:
owner[owner.start_date > '2022-02-22'].owner.value_counts()